In [1]:
import numpy as np
import pandas as pd
import pickle
import time
import itertools

import matplotlib
matplotlib.rcParams.update({'font.size': 17.5})

import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rc('axes.formatter', useoffset=False)


import sys
import os.path
sys.path.append( os.path.abspath(os.path.join( os.path.dirname('..') , os.path.pardir )) )

In [2]:
#import FLAME with reuse of control units
from colFLAMEbit_c import *
from FLAMEbit_c import *

In [3]:
def construct_sec_order(arr):
    ''' an intermediate data generation function used 
        for generating second order information '''
    
    second_order_feature = []
    num_cov_sec = len(arr[0])
    for a in arr:
        tmp = []
        for i in range(num_cov_sec):
            for j in range(i+1, num_cov_sec):
                tmp.append( a[i] * a[j] )
        second_order_feature.append(tmp)
        
    return np.array(second_order_feature)

In [4]:
def data_generation_hybrid(num_control, num_treated, num_cov, exponential = True):
    
    # a data generation function, not used here
    
    xc = np.random.binomial(1, 0.5, size=(num_control, num_cov))   # data for conum_treatedrol group
    xt = np.random.binomial(1, 0.5, size=(num_treated, num_cov))   # data for treatmenum_treated group
        
    errors1 = np.random.normal(0, 0.05, size=num_control)    # some noise
    errors2 = np.random.normal(0, 0.05, size=num_treated)    # some noise
    
    #dense_bs_sign = np.random.choice([-1,1], num_cov_dense)
    if exponential:
        dense_bs = [ 20.*((4./5)**(i+1)) for i in range(num_cov) ]
    else:
        dense_bs = [ (20./(i+1)) for i in range(num_cov) ]

    yc = np.dot(xc, np.array(dense_bs)) #+ errors1     # y for control group 

    # y for treated group 
    treatment_eff_coef = np.random.normal( 1.5, 0.05, size=num_cov) #beta
    treatment_effect = np.dot(xt, treatment_eff_coef) 
    
    yt = np.dot(xt,np.array(dense_bs))+treatment_effect 
                                      # + errors2    
        
    df1 = pd.DataFrame(np.hstack([xc]), 
                       columns=range(num_cov))
    df1['outcome'] = yc
    df1['treated'] = 0

    df2 = pd.DataFrame(np.hstack([xt]), 
                       columns=range(num_cov ) ) 
    df2['outcome'] = yt
    df2['treated'] = 1

    df = pd.concat([df1,df2])
    df['matched'] = 0
  
    return df, dense_bs, treatment_eff_coef

In [20]:
# data generation, set exponential to be True or False for exponential decay and power-law decay respectively

d = data_generation_hybrid(40000, 2000, 18, exponential=True)
df = d[0] 
holdout,_,_ = data_generation_hybrid(40000, 2000, 18, exponential=True)

In [7]:
#------- EXPERIMENT 1:RATIO 1 -----------------------------#

In [17]:
#get the dataframe with different ratio of treatments/controls
df1 = df.iloc[20000:42000,:] 
df2 = df.iloc[30000:42000,:] 
holdout1 = holdout.iloc[20000:42000,:] 
holdout2 = holdout.iloc[30000:42000,:] 

In [18]:
res_gen = run_bit(df, holdout, range(18), [2]*18, threshold = -10, tradeoff_param = 0.001)

1714
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
-0.974467791601
1460
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
-1.96349141636
1082
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
-3.15532875899
578
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
-4.37367651093
156
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
-5.91597205422
21
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
-8.32306463632
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
no more matches


In [19]:
res = run_mpbit(df, holdout, range(18), [2]*18, threshold =-10, tradeoff_param = 0.001)

1714
1460
1246
1078
924
782
670
566
475
410
344
300
252
219
178
156
141
118
96
77
62
50
46
42
38
34
27
23
21
17
15
10
8
7
7
6
6
5
4
4
4
4
4
4
3
2
2
1
0
no more matches


In [20]:
pickle.dump(res_gen, open('413gen-ratio1', 'wb'))
pickle.dump(res, open('413col-ratio1', 'wb'))
pickle.dump(d, open('413data-ratio1', 'wb'))

df.to_csv("413dataratio1.csv")


In [21]:
def ground_truth( eff_coef, covs_ordered, num_covs_dense = 18, num_second_order = 0, second_order = True):
    arr = np.array(list(itertools.product([0,1], repeat=num_covs_dense)))
    effect = np.dot(arr, eff_coef)
    if second_order:
        second_effect = np.sum(construct_sec_order(arr[:,:num_second_order] ), axis=1)
        effect = effect + second_effect
    df = pd.DataFrame(arr, columns=covs_ordered)
    df['effect'] = effect
    return df



In [22]:
ground_truth = ground_truth(d[2], list(range(18)), num_covs_dense = 18, num_second_order = 0, second_order = False)

ground_truth.to_csv("413groundtruthratio1.csv")


In [23]:
truth_list = []
pred_list = []
count = 0
av_err_cate = []
aux_size = []
for r in res[1]:
    count = count +1
    tmp = pd.merge(r, ground_truth, on = list(set(range(18)) & set(r.columns) ), how = 'left')
    truth_list = truth_list + list(tmp['effect_y'])
    pred_list = pred_list + list(tmp['effect_x'])
    aux_size = aux_size + list(tmp['size'])



In [24]:
# generate the lists of true and estimated cates for genFLAME

truth_list_gen = []
pred_list_gen = []

aux_size_gen = []
av_err_cate_gen = []
for r_gen in res_gen[1]:
    tmp_gen = pd.merge(r_gen, ground_truth, on = list(set(range(18)) & set(r_gen.columns) ), how = 'left')
    truth_list_gen = truth_list_gen + list(tmp_gen['effect_y'])
    pred_list_gen = pred_list_gen + list(tmp_gen['effect_x'])
    aux_size_gen = aux_size_gen + list(tmp_gen['size'])



In [25]:
# create a dataframe with the true and estimated cates : 


effect_col = pd.DataFrame()
effect_col['pred'] = pred_list
effect_col['true'] = truth_list
effect_col['size'] = aux_size

effect_col['method'] = ['collapsing FLAME']*len(truth_list)

effect_gen = pd.DataFrame()
effect_gen['pred'] = pred_list_gen
effect_gen['true'] = truth_list_gen
effect_gen['method'] = ['generic FLAME']*len(truth_list_gen)
effect_gen['size'] = aux_size_gen


effect = pd.concat([effect_gen, effect_col])
#effect_gen.to_csv('effect_gen.csv')

effect_col.to_csv('effect413col_ratio1.csv')

effect_gen.to_csv('effect413gen_ratio1.csv')

effect.to_csv('effect413_ratio1.csv')


In [ ]:
import seaborn as sns
sns.lmplot(x="true", y="pred",hue="method", data = effect, fit_reg=False)


In [ ]:
#------Ratio2---#

In [28]:
res_gen1 = run_bit(df1, holdout1, range(18), [2]*18, threshold =-10, tradeoff_param = 0.001)


1857
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
-0.974566384023
1719
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
-1.96419333275
1499
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
-3.15602338887
1106
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
-4.37309878157
579
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
-5.91744665272
163
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
-8.31804840249
19
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
-10.9958662654
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
no more matches


In [29]:
res1 = run_mpbit(df1, holdout1, range(18), [2]*18, threshold = -10, tradeoff_param = 0.001)

1857
1719
1602
1470
1358
1255
1172
1073
990
910
847
782
725
669
619
586
541
494
458
412
379
349
322
300
283
265
242
229
218
201
189
171
156
149
136
132
122
112
102
91
88
83
74
69
62
56
54
46
41
36
35
32
30
28
24
18
18
17
15
13
12
11
10
8
8
7
7
7
7
7
7
6
6
5
5
5
5
5
4
4
4
4
4
4
4
4
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
0
no more matches


In [30]:
pickle.dump(res_gen1, open('413gen-ratio2', 'wb'))
pickle.dump(res1, open('413col-ratio2', 'wb'))

df1.to_csv("413dataratio2.csv")


In [31]:
truth_list1 = []
pred_list1 = []
count = 0
av_err_cate = []
aux_size1 = []
for r in res1[1]:
    count = count +1
    tmp1 = pd.merge(r, ground_truth, on = list(set(range(18)) & set(r.columns) ), how = 'left')
    truth_list1 = truth_list1 + list(tmp1['effect_y'])
    pred_list1 = pred_list1 + list(tmp1['effect_x'])
    aux_size1 = aux_size1 + list(tmp1['size'])


truth_list_gen1 = []
pred_list_gen1 = []

aux_size_gen1 = []
av_err_cate_gen1 = []
for r_gen in res_gen1[1]:
    tmp_gen1 = pd.merge(r_gen, ground_truth, on = list(set(range(18)) & set(r_gen.columns) ), how = 'left')
    truth_list_gen1 = truth_list_gen1 + list(tmp_gen1['effect_y'])
    pred_list_gen1 = pred_list_gen1 + list(tmp_gen1['effect_x'])
    aux_size_gen1 = aux_size_gen1 + list(tmp_gen1['size'])




In [32]:
# create a dataframe with the true and estimated cates : 

effect_col1 = pd.DataFrame()
effect_col1['pred'] = pred_list1
effect_col1['true'] = truth_list1
effect_col1['size'] = aux_size1

effect_col1['method'] = ['collapsing FLAME']*len(truth_list1)


#effect_col.to_csv('effect.csv')

effect_gen1 = pd.DataFrame()
effect_gen1['pred'] = pred_list_gen1
effect_gen1['true'] = truth_list_gen1
effect_gen1['method'] = ['generic FLAME']*len(truth_list_gen1)
effect_gen1['size'] = aux_size_gen1


effect1 = pd.concat([effect_gen1, effect_col1])


effect_col1.to_csv('effect413col_ratio2.csv')

effect_gen1.to_csv('effect413gen_ratio2.csv')

effect1.to_csv('effect413_ratio2.csv')

In [ ]:
import seaborn as sns
sns.lmplot(x="true", y="pred",hue="method", data = effect1, fit_reg=False)


In [38]:
#--RATIO3---#

In [35]:
res_gen2 = run_bit(df2, holdout2, range(18), [2]*18, threshold =-10, tradeoff_param = 0.001)


1919
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
-0.974696177791
1845
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
-1.96383269351
1719
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
-3.15490115698
1485
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
-4.3725634571
1108
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
-5.91487081324
604
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
-8.32286640009
180
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
-11.0118542525
30
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
-14.4531214798
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
no more matches


In [36]:

res2 = run_mpbit(df2, holdout2, range(18), [2]*18, threshold = -10, tradeoff_param = 0.001)



1919
1845
1778
1701
1638
1578
1524
1464
1415
1358
1314
1262
1221
1177
1123
1080
1040
1006
968
933
903
858
825
797
770
742
712
692
665
648
628
598
571
558
541
524
507
493
469
455
445
430
412
400
383
367
358
339
332
314
304
295
288
275
263
253
243
236
225
213
200
190
180
169
163
155
151
144
137
132
128
121
117
111
108
102
95
92
88
85
83
81
77
74
72
72
72
70
68
66
63
61
61
58
56
55
51
50
48
47
43
42
41
41
41
40
39
37
35
35
35
35
34
33
31
31
29
29
28
26
26
25
24
24
23
20
20
19
19
18
18
18
18
18
18
17
17
17
16
15
14
13
13
13
11
11
11
11
11
11
11
11
10
10
10
10
10
9
9
9
9
8
8
8
8
7
7
7
7
6
5
5
5
5
5
5
5
5
5
5
4
4
4
4
4
4
4
4
3
3
3
3
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
no more matches


In [37]:
pickle.dump(res_gen2, open('413gen-ratio3', 'wb'))
pickle.dump(res2, open('413col-ratio3', 'wb'))

df2.to_csv("413dataratio3.csv")


In [38]:
truth_list2 = []
pred_list2 = []
count = 0
av_err_cate = []
aux_size2 = []
for r in res2[1]:
    count = count +1
    tmp2 = pd.merge(r, ground_truth, on = list(set(range(18)) & set(r.columns) ), how = 'left')
    truth_list2 = truth_list2 + list(tmp2['effect_y'])
    pred_list2 = pred_list2 + list(tmp2['effect_x'])
    aux_size2 = aux_size2 + list(tmp2['size'])


truth_list_gen2 = []
pred_list_gen2 = []

aux_size_gen2 = []
for r_gen in res_gen2[1]:
    tmp_gen2 = pd.merge(r_gen, ground_truth, on = list(set(range(18)) & set(r_gen.columns) ), how = 'left')
    truth_list_gen2 = truth_list_gen2 + list(tmp_gen2['effect_y'])
    pred_list_gen2 = pred_list_gen2 + list(tmp_gen2['effect_x'])
    aux_size_gen2 = aux_size_gen2 + list(tmp_gen2['size'])

 





In [39]:

# create a dataframe with the true and estimated cates : 


effect_col2 = pd.DataFrame()
effect_col2['pred'] = pred_list2
effect_col2['true'] = truth_list2
effect_col2['size'] = aux_size2

effect_col2['method'] = ['collapsing FLAME']*len(truth_list2)

effect_gen2 = pd.DataFrame()
effect_gen2['pred'] = pred_list_gen2
effect_gen2['true'] = truth_list_gen2
effect_gen2['method'] = ['generic FLAME']*len(truth_list_gen2)
effect_gen2['size'] = aux_size_gen2


effect2 = pd.concat([effect_gen2, effect_col2])


effect_col2.to_csv('effect413col_ratio3.csv')

effect_gen2.to_csv('effect413gen_ratio3.csv')

effect2.to_csv('effect413_ratio3.csv')

In [ ]:
import seaborn as sns
sns.lmplot(x="true", y="pred",hue="method", data = effect2, fit_reg=False)
